In [ ]:
!pip install retina-face

In [8]:
from retinaface import RetinaFace
import matplotlib.pyplot as plt
import cv2
import os

In [3]:
def load_face(folder):
    faces = []
    for path in os.listdir(folder):
        face = RetinaFace.extract_faces(os.path.join(folder, path), align=True)
        faces.append(face[0])

    return faces

In [4]:
# detect face
list_face = []
for i in range(0, 14):
    folder = "/content/drive/MyDrive/Colab Notebooks/dataface/img%d" %i
    list_face.append(load_face(folder=folder))

Directory  /root /.deepface created
Directory  /root /.deepface/weights created
retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5
100%|██████████| 119M/119M [00:00<00:00, 191MB/s]


In [5]:
import numpy as np
print(np.array(list_face).shape)

(14,)


In [ ]:
!pip install arcface

In [10]:
from arcface import ArcFace

In [11]:
face_rec = ArcFace.ArcFace()

In [9]:
# list of embedding vectors
list_emb = []

for j in range(0, 14):
    embs = []
    for face in list_face[j]:
        embs.append(face_rec.calc_emb(face))

    list_emb.append(embs)

In [21]:
print(np.array(list_emb).shape)

(14,)


In [2]:
import numpy as np

In [22]:
# save embedding vectors
with open("/content/drive/MyDrive/Colab Notebooks/dataface/list_emb.npy", 'wb') as f:
    np.save(f, np.array(list_emb))

In [3]:
# load embedding vectors
with open("/content/drive/MyDrive/Colab Notebooks/dataface/list_emb.npy", 'rb') as f:
    list_emb = np.load(f, allow_pickle=True)

print(list_emb.shape)

(14,)


In [4]:
def min_embs(emb_test, embs):
    min = 1000
    for emb in embs:
        dis = face_rec.get_distance_embeddings(emb_test, emb)
        if(dis < min):
            min = dis

    return min

In [13]:
folder_test = "/content/drive/MyDrive/Colab Notebooks/dataface/img_test"

list_path = []
faces_test = []
for path in os.listdir(folder_test):
    face = RetinaFace.extract_faces(os.path.join(folder_test, path), align=True)
    faces_test.append(face)
    list_path.append(path)

# faces_test = load_face(folder=folder_test)

embs_test = []
for face in faces_test:
    embs_test.append(face_rec.calc_emb(face))    # list of embedding test


In [14]:
# detect label

list_label = []
list_min_dis = []
for emb in embs_test:
    min_emb = 1000
    min_label = -1
    for i in range(0, 14):
        min = min_embs(emb, list_emb[i])    # get min of distance between emb_test and list of embedding in class i
        if(min < min_emb):
            min_emb = min
            min_label = i

    list_min_dis.append(min_emb)
    if(min_emb > 0.5):          # threshold: 0.5
        list_label.append(-1)   # unknow
    else:
        list_label.append(min_label)

In [15]:
print(list_path)
print(list_label)
print(list_min_dis)

['4_frame72.jpg', '9_frame34.jpg', '11_frame29.jpg', '12_frame74.jpg', '13_frame68.jpg', 'img3.jpg', 'img5.jpg', 'img6.jpg', 'img8.jpg', '12_frame105.jpg', '6_frame14.jpg']
[4, 9, 11, 12, 13, -1, -1, -1, -1, 12, 6]
[0.10257189, 0.16308057, 0.24654679, 0.13014537, 0.3386658, 0.9120693, 1.0613893, 1.1343672, 1.0421782, 0.29990458, 0.0]


In [ ]:
list_min = []
for i in range(0, 14):
    min = min_embs(emb_test, list_emb[i])   # max embedding of class i
    list_min.append(min)

In [ ]:
from numpy import dot
from numpy.linalg import norm

def cosine_vec(a, b):
    return dot(a, b)/(norm(a)*norm(b))

In [ ]:
def min_cosine(emb_test, embs):
    min = 1000
    for emb in embs:
        dis_cos = cosine_vec(emb, emb_test)
        if(dis_cos < min):
            min = dis_cos
    
    return min

In [ ]:
list_cosine = []
for i in range(0, 14):
    min = min_cosine(emb_test, list_emb[i])
    list_cosine.append(min)